# Parametrización del Algoritmo de Lobo Gris Discreto (DGWO) para su aplicación en la Distribución de Instalaciones con Áreas Desiguales (UAFLP)

El proceso de *Parametrización* o tuneo de los parámetros del DGWO para su aplicación en problemas UAFLP consiste en experimentar diferentes valores de los parámetros del algoritmo (p.e., diferentes valores de tamaño de la manada) con el fin de conocer el conjunto de parámetros que mejor se adapta (es decir, que obtiene la mejor solución) para la solución de los problemas UAFLP, previo a su aplicación al caso de estudio real.

Para ello, el DGWO se aplicará a las siguientes instancias de datos recolectadas de la literatura relacionada con el UAFLP:

| Instancia | Num Dptos | Lados de Instalación ($W x H$) | Tasa de Aspecto | Norma de Distancia | Referencia              |
|-----------|-----------|--------------------------------|-----------------|--------------------|-------------------------|
| O7        |     7     |        ($8.54$ x $13.00$)      |   $\alpha = 4$  |     Rectilínea     | Meller et al. (1998)    |
| O8        |     8     |       ($11.31$ x $13.00$)      |   $\alpha = 4$  |     Rectilínea     | Meller et al. (1998)    |
| O9        |     9     |       ($12.00$ x $13.00$)      |   $\alpha = 4$  |     Rectilínea     | Meller et al. (1998)    |
| vC10R-a   |    10     |       ($25.00$ x $51.00$)      |   $\alpha = 5$  |     Rectilínea     | van Camp et al. (1992)  |
| Garment   |    11     |       ($25.00$ x $45.60$)      |   $\alpha = 4$  |     Rectilínea     | Zapata et al. (2023)    |
| MB12      |    12     |        ($6.00$ x $8.00$)       |   $\alpha = 4$  |     Rectilínea     | Bozer and Meller (1997) |

**Referencias**:

* Bozer, Y.A., Meller, R.D. (1997). A reexamination of the distance-based facility layout problem. IIE Trans., 29. pp. 549-560
* Meller, R.D., Narayanan, V., Vance, P.H. (1998). Optimal facility layout design. Oper. Res. Let., 23. pp. 117-127
* van Camp, D.J., Carter, M.W., Vannelli, A. (1992). A nonlinear optimization approach for solving facility layout problems. Eur. J. Oper. Res., 57. pp. 174-189 
* Zapata-Cortés, J.A., Arango-Serna, M.D., Cáceres-Gelvez, S. (2023). Genetic algorithm for the optimization of the Unequal-Area Facility Layout Problem. In: Handbook on Decision Making, 3. pp. 399-418

El algoritmo DGWO incluye los siguientes parámetros, los cuales se buscarán ajustar para la optimización del UAFLP:

* `tam_manada` ($M$): Hace referencia al tamaño de la manada, o el número de individuos (lobos) que comprenden la manada dentro del algoritmo.
* `n_lideres` ($N$): Hace referencia al número de lobos líderes, los cuales guiarán la búsqueda la presa (mejor solución). Solo puede tomar los valores `1` ($\alpha$) y `3` ($\alpha, \beta y \delta$). Los demás lobos son tratados como lobos de la manada que seguirán a uno de los líderes en los comportamientos de búsqueda y ataque.
* `theta_1` ($\theta_1$): Hace referencia al porcentaje del tamaño del movimiento que debe realizarse durante el comportamiento de búsqueda de la presa. Es un parámetro que debe estar entre `0` y `1`.
* `theta_2` ($\theta_2$): Hace referencia al porcentaje de la distancia mínima que debe tener cada lobo en la manada con respecto a uno de los líderes. Es un parámetro que debe estar entre `0` y `1`.
* `theta_3` ($\theta_3$): Hace referencia a la probabilidad de seleccionar al lobo alfa ($\alpha$) entre los lobos líderes para reducir la distancia de los lobos en la manada. Es un parámetro que debe estar entre `0` y `1`, y debe ser igual a `1` si `n_lideres = 1`.

Las instancias de datos se extraen de archivos de datos a través de la función `obtener_datos` de la clase `ModeloUAFLP` y el algoritmo DGWO será utilizado, a través de la clase `AlgLoboGris`, para optimizar estas instancias usando los siguientes valores de los parámetros:

* `tam_manada` ($M$): `[6, 8, 10, 12]`
* `n_lideres` ($N$): `[1, 3]`
* `theta_1` ($\theta_1$): `[0.2, 0.4, 0.6, 0.8]`
* `theta_2` ($\theta_2$): `[0.2, 0.4, 0.6, 0.8]`
* `theta_3` ($\theta_3$): `[0.2, 0.4, 0.6, 0.8]` o `[1]` si `n_lideres = 1`

## 1. Clase `ModeloUAFLP`

Clase para el problema UAFLP: